# Data 전처리(Data Preprocessing)

In [1]:
# 전처리를 위해 필요한 라이브러리 - 머신러닝과 관련됨
# !pip install scikit-learn

     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     --------------------------------- ------ 51.2/60.6 kB 2.6 MB/s eta 0:00:01
     ---------------------------------------- 60.6/60.6 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/10.6 MB 33.6 MB/s eta 0:00:01
   -------------------- ------------------- 5.4/10.6 MB 69.4 MB/s eta 0:00:01
   -------------------------------------- - 10.1/10.6 MB 80.9 MB/s eta 0:00:01
   ---------------------------------------- 10.6/10.6 MB 93.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/302.2 kB ? eta -:--:--
   ---------------------------------------- 302.2/302.2 kB ? eta 0:00:00
   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   --- ------------------------------------ 4.5/46.2 MB 94.8 MB/s eta 0:00:01
   ----- ---------------------------------- 6.3/46.2 MB 101.2 MB/s eta 0:00:01
   ----- ----

## 결측치(Missing Value) 처리
- 결측치 : 수집하지 못한 값
- 결측치를 처리하기 전에 "기록되지 않은건지", "존재하지 않은건지"를 확인해야 함

### 결측치 처리 방법
- 결측치 삭제
  - 리스트와이즈 삭제 - 결측치가 있는 행들을 삭제 => 데이터가 많을 때 적합
  - 컬럼 삭제 - 속성하나가 날라가므로 타격이 큼. 이런 경우는 많이 없다
  - 페어와이즈 삭제 - 분석할때는 사용하지만 머신러닝에서는 사용 X. 실행시점에 삭제함


In [2]:
import pandas as pd
import numpy as np
data = {
    "name":['김영희', '이명수', '박진우', '이수영', '오영미'],
    "age": [23, 18, 25, 32, np.nan], 
    "weight":[np.nan, 80, np.nan, 57, 48]
}
df = pd.DataFrame(data)
df

,name,age,weight
0,김영희,23.0,NaN
1,이명수,18.0,80.0
2,박진우,25.0,NaN
3,이수영,32.0,57.0
4,오영미,NaN,48.0


In [5]:
# 결측치 확인 - 전체
df.isna().sum()  # 컬럼별 결측치 개수

name      0
age       1
weight    2
dtype: int64

In [6]:
df.isna().sum(axis=1) # 행별로 결측치 개수

0    1
1    0
2    1
3    0
4    1
dtype: int64

In [9]:
# 결측치 확인 - Series
df['name'].isna().sum()

0

In [16]:
# 조회한 원소가 결측치인지
pd.isna(df.loc[2, 'weight'])  # isna() 함수: arg 값이 결측치인지

True

In [17]:
# 제거 - 행단위(리스트와이즈)
df.dropna(axis=0)

,name,age,weight
1,이명수,18.0,80.0
3,이수영,32.0,57.0


In [18]:
# 컬럼단위
df.dropna(axis=1)

,name
0,김영희
1,이명수
2,박진우
3,이수영
4,오영미


In [20]:
## pair wise 삭제 - 나이와 키간의 상관관계를 분석
### 상관관계를 분석 결과: 상관계수. -1 ~ +1 사이의 실수
### 양수(0~1): 양의 상관관계. 비례관계
### 음수(-1~0): 음의 상관관계. 반비례관계
### 0: 관계없
df[['age', 'weight']].dropna().corr()

,age,weight
1,18.0,80.0
3,32.0,57.0


### 결측치 대체(imputation)
- 그 값일 가능성이 가장 높은 값으로 대체
- 여러방법이 있는데 **K-최근접 이웃(K-NN) 대체**가 많이 쓰임

In [23]:
df.fillna(100)  # 전체 NA를 100으로 대체

,name,age,weight
0,김영희,23.0,100.0
1,이명수,18.0,80.0
2,박진우,25.0,100.0
3,이수영,32.0,57.0
4,오영미,100.0,48.0


In [28]:
import pandas as pd
import numpy as np

df = pd.DataFrame(
    [
        [1, 2, np.nan],
        [3, 4, 3], 
        [np.nan, 6, 5],
        [8, np.nan, 7],
        [1, 2, 5],
        [np.nan, 1, 7]
    ], columns=['A', 'B', 'C']
)
org = df.copy()

In [39]:
# 컬럼별(속성)처리
###  평균 대체
df['A'] = df['A'].fillna(df['A'].mean())
###  중앙값
df['B'] = df['B'].fillna(df['B'].median())
###  최빈값(범주형)
df['C'] = df['C'].fillna(df['C'].mode()[0]) # mode(): Series를 반환

In [40]:
df

,A,B,C
0,1.00,2.0,5.0
1,3.00,4.0,3.0
2,3.25,6.0,5.0
3,8.00,2.0,7.0
4,1.00,2.0,5.0
5,3.25,1.0,7.0


In [43]:
### scikit-learn 패키지를 이용해서 전처리
from sklearn.impute import SimpleImputer

df=org.copy()
# 객체 생성 -> 어떤값으로 변경할지를 설정
imputer = SimpleImputer(strategy="median")
# strategy="median":중앙값, "mean":평균, "most_frequent":최빈값, "constant":상수(fill_value=채울값)
result = imputer.fit_transform(df)
# 변환. axis=0 기준으로 strategy 연산한 값으로 결측치를 대처함. 처리결과: ndarray(numpy)
result

array([[1., 2., 5.],
       [3., 4., 3.],
       [2., 6., 5.],
       [8., 2., 7.],
       [1., 2., 5.],
       [2., 1., 7.]])

In [45]:
result_df = pd.DataFrame(result, columns=df.columns)
result_df

,A,B,C
0,1.0,2.0,5.0
1,3.0,4.0,3.0
2,2.0,6.0,5.0
3,8.0,2.0,7.0
4,1.0,2.0,5.0
5,2.0,1.0,7.0


In [49]:
df = org.copy()
# "A", "B": 중앙값  "C": 평균
imputer1 = SimpleImputer(strategy="median")
imputer2 = SimpleImputer(strategy="mean")
# fit(): 변환할 때 필요한 값들을 찾아서 instance변수에 저장. (컬럼별 평균, 중앙값)
# transform(): 변환작업
# fit_transform(): fit(), transform()을 순서대로 한번에 처리
result1 = imputer1.fit_transform(df[['A', 'B']])
result2 = imputer2.fit_transform(df['C'].to_frame()) # 변환대상은 행/열의 2차원으로 입력해야한다 (6,) -> (6,1)
# print(result1)
# print(result2)
result_df = pd.concat([pd.DataFrame(result1), pd.DataFrame(result2)], axis=1)
result_df.columns = df.columns
result_df

,A,B,C
0,1.0,2.0,5.4
1,3.0,4.0,3.0
2,2.0,6.0,5.0
3,8.0,2.0,7.0
4,1.0,2.0,5.0
5,2.0,1.0,7.0


In [51]:
# 2차원 형태로 넣어줘야한다. 그래서 시리즈일때는 데이터프레임형태로 바꿔서 넣어주기
df['C'].shape # 시리즈
df['C'].to_frame().shape # 데이터프레임

(6, 1)

In [52]:
# KNNImputer 이용해 결측치를 대체
## 머신러닝 알고리즘 중 K-NN을 이용 (K 최근접(Nearest) 이웃(Neighbors))
from sklearn.impute import KNNImputer

df = org.copy()
df

,A,B,C
0,1.0,2.0,NaN
1,3.0,4.0,3.0
2,NaN,6.0,5.0
3,8.0,NaN,7.0
4,1.0,2.0,5.0
5,NaN,1.0,7.0


In [53]:
imputer = KNNImputer(n_neighbors=1) # 가장 가까운값 몇개(1)를 이용해서 결측치를 찾을지
result = imputer.fit_transform(df)
result_df = pd.DataFrame(result, columns=df.columns)
result_df

# 결측치를 제외한 나머지 값들이 가까운 값을 찾는

,A,B,C
0,1.0,2.0,5.0
1,3.0,4.0,3.0
2,3.0,6.0,5.0
3,8.0,1.0,7.0
4,1.0,2.0,5.0
5,8.0,1.0,7.0


## 이상치(Outlier) 처리
- 이상치는 컬럼별로 본다 -> 나이와 키가 있으면 나이는 나이별로 보고 키는 키별로 본다


### 이상치 식별
- 통계적 기준
  - 표준편차 기준: 평균으로부터 K 표준편차 범위 밖으로 떨어진 데이터를 outlier로 판단
  - 분위수 기준: IQR을 이용해 Outlier여부를 찾는다
    - 1분위, 3분위에서 IQR * 1.5보다 더 떨어진 값을 outlier로 판단한다. 1.5는 변경가능
  - 극단치(분포에서 벗어난 값): 정상적인 값이지만 다른 패턴을 가지는 값. 일반적으로 극단적으로 크거나 작은 값

In [54]:
import pandas as pd
import numpy as np
np.random.seed(0)
# 평균:10, 표준편차:2 기준의 정규분포를 따르는 난수를 생성. 10행,3열 형태로 생성.
df = pd.DataFrame(np.random.normal(10, 2, size=(10, 3)), columns=['a', 'b', 'c'])
df.iloc[[0, 3], [0, 2]] = [[100, 200],[300,-100]]
df

,a,b,c
0,100.000000,10.800314,200.000000
1,14.481786,13.735116,8.045444
2,11.900177,9.697286,9.793562
3,300.000000,10.288087,-100.000000
4,11.522075,10.243350,10.887726
5,10.667349,12.988158,9.589683
6,10.626135,8.291809,4.894020
7,11.307237,11.728872,8.515670
8,14.539509,7.091269,10.091517
9,9.625632,13.065558,12.938718


In [55]:
df.iloc[[0,3], [0,2]]

,a,c
0,100.0,200.0
3,300.0,-100.0


In [57]:
# 분위수를 기준으로 outlier를 찾기(식별)
### 1분위, 3분위계산......
q1, q3 = df['a'].quantile(q=[0.25, 0.75]) # 시리즈 대입
iqr = q3-q4 # IQR: 1분위 ~ 3단위격 크기
iqr *= q3=q1

0.25    10.827321
0.75    14.525079
Name: a, dtype: float64

In [60]:
# outlier계산 함수
def is_outlier(column, whis=1.5):
    # whis(whisker): 정상범위를 조정하는 값. 기본값 1.5 범위를 넓힐 경우 더 큰값
    # 1,3분위수 계산
    q1, q3 = column.quantile(q=[0.25, 0.75])
    iqr = (q3 - q1) * whis # IQR의 whis 배 한 값
    # 원소별로 outlier인지 체크
    return ~column.between(q1-iqr, q3+iqr)

In [61]:
is_outlier(df['a'])

0     True
1    False
2    False
3     True
4    False
5    False
6    False
7    False
8    False
9    False
Name: a, dtype: bool

In [62]:
result = df.apply(is_outlier)
result

,a,b,c
0,True,False,True
1,False,False,False
2,False,False,False
3,True,False,True
4,False,False,False
5,False,False,False
6,False,False,False
7,False,False,False
8,False,False,False
9,False,False,False


In [63]:
result.sum()

a    2
b    0
c    2
dtype: int64

In [65]:
items = ['TV', '냉장고', '컴퓨터', '컴퓨터', '냉장고', '에어콘',  'TV', '에어콘']

# labelEncoder를 이용해서 인코딩할 때 입력값은 1차원 자료구조
import numpy as np
from sklearn.preprocessing import LabelEncoder

# LabelEncoder의 instance 생성
le = LabelEncoder()
# 학습(fit) -> 각 고유값들을 찾고 그 고유값을 어떤 정수로 바꿀지 계산
le.fit(items)
# 변환(transform()) -> 학습 결과에 맞춰서 값들을 변환
result1 = le.transform(items) # 처리결과: ndarray(numpy 자료구조)
print(result1)

[0 1 3 3 1 2 0 2]


In [76]:
# 어떻게 변환했는지 fit(학습)한 결과를 조회

print(le.classes_) # 값: 고유값, index: encoding 

['TV' '냉장고' '에어콘' '컴퓨터']


In [77]:
le.classes_[result1]

array(['TV', '냉장고', '컴퓨터', '컴퓨터', '냉장고', '에어콘', 'TV', '에어콘'], dtype='<U3')

In [81]:
# fit 대상과 transform 대상이 동일한 경우 -> fit_transform() 한번에 변환
le2 = LabelEncoder()
result2 = le2.fit_transform(items)
result2

array([0, 1, 3, 3, 1, 2, 0, 2], dtype=int64)

In [82]:
## encoding된 것을 decoding (정수 -> 원래 범주값)
le2.inverse_transform([1, 1, 1, 2, 2])

array(['냉장고', '냉장고', '냉장고', '에어콘', '에어콘'], dtype='<U3')

In [86]:
## 학습대상과 변환대상이 다른경우
le3 = LabelEncoder()
le3.fit(['TV', '에어콘', '컴퓨터', '냉장고', '노트북', '공기청정기', '청소기'])
le3.classes_

array(['TV', '공기청정기', '냉장고', '노트북', '에어콘', '청소기', '컴퓨터'], dtype='<U5')

In [87]:
le3.transform(items)

array([0, 2, 6, 6, 2, 4, 0, 4])

In [88]:
cols = ['age', 'workclass','fnlwgt','education', 'education-num', 'marital-status', 'occupation','relationship', 'race', 'gender','capital-gain','capital-loss', 'hours-per-week','native-country', 'income']

In [91]:
import pandas as pd

data = pd.read_csv('data/adult.data', 
                   header=None,   # 컬럼명이 없다. 첫번째줄부터 데이터 행
                   names=cols,    # 컬럼명 설정
                   na_values='?', # ?를 결측치로 읽어라
                   skipinitialspace=True 
                  )
data.shape

(32561, 15)

In [92]:
## 결측치 제거
data.dropna(inplace=True)
data.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
gender            0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64